# Time Series Forecasting with Autoregressive GPT with FFT

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import os
import random
import numpy as np
import pandas as pd
import math
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['axes.grid'] = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'USING DEVICE: {device}')

# Hyperparameters

In [ ]:
features_with_fft = ['nat_demand'] # TODO: add features here that require FFT

hyperparameters = {
    'window_size': 100, # also 'context_size'
    'features_with_fft': features_with_fft,
    'input_features_size': 16 + len(features_with_fft),
    'date_input_features_size': 3, # (MONTH, DAY, HOUR)
    'date_features_dim': 64,
    'hidden_features_size': 256-64, # will be concat inside model
    'output_features_size': 16 + len(features_with_fft),
    'num_heads': 4,
    'ff_dim': 256*4, # usually 4 times the hidden feature size
    'num_decoder_layers': 12,
    'emb_dropout_prob': 0.1,
    'attn_dropout_prob': 0.1,
    'ff_dropout_prob': 0.1,
    'attn_use_bias': False,
    'ff_use_bias': False,
    'output_features_bias': False,
    'batch_size': 128,
    'split_ratio': 0.8, # 80% training, 20% testing
    'learning_rate': 0.001,
    'num_epochs': 60,
    'use_amp': True, # USE MIXED PRECISION
}

# Dataset

In [ ]:
# Airline Passgeners
#df_full = pd.read_csv('data/airline_passengers/airline-passengers.csv')

# Panama Electricity Load Forecasting
df_full = pd.read_csv('data/panama_electricity_load_forecasting/train.csv')

df_full.head()

In [ ]:
# Airline Passgeners
#df_full['Month'] = pd.to_datetime(df_full['Month'])

# Panama Electricity Load Forecasting
df_full['datetime'] = pd.to_datetime(df_full['datetime'], dayfirst=True)

df_full.head()

In [ ]:
# # Airline Passgeners
#df_full.set_index('Month', inplace=True)

# Panama Electricity Load Forecasting
df_full.set_index('datetime', inplace=True)

df_full.tail()

In [ ]:
# Airline Passgeners
#df_full.plot()

# Panama Electricity Load Forecasting
df_full.loc['2019-01-03 05:00:00':'2019-01-12 05:00:00', 'nat_demand'].plot()

# Standart Scaler
* Exercise: Try without scaler, see if learning works!

In [ ]:
feature_scaler = StandardScaler()

# Airline Passgeners
#df_full['Passengers'] = scaler.fit_transform(df_full['Passengers'].values.reshape(-1, 1))

# Panama Electricity Load Forecasting
df_full[df_full.columns] = feature_scaler.fit_transform(df_full[df_full.columns])

# Airline Passgeners
#df_full.plot()

# Panama Electricity Load Forecasting
df_full.loc['2019-01-03 05:00:00':'2019-01-12 05:00:00', 'nat_demand'].plot()

In [ ]:
tstamp = df_full.index[-1]
tstamp

# Time series data shape: 
Unbatched: $(S, F)$

* $S:$ Sequence Length
* $F:$ Number of Features

Batched: $(B, S, F)$
* $B:$ Batch Size

In [ ]:
df_full.shape

# Model

In [ ]:
from models.transformer import GPTTimeSeries

## Transformer Model
### Initialized Model with Hyperparameters

In [ ]:
model = GPTTimeSeries(
    input_features_size=hyperparameters['input_features_size'],
    date_input_features_size=hyperparameters['date_input_features_size'],
    date_features_dim=hyperparameters['date_features_dim'],
    features_dim=hyperparameters['hidden_features_size'],
    output_features_size=hyperparameters['output_features_size'],
    num_heads=hyperparameters['num_heads'],
    ff_dim=hyperparameters['ff_dim'],
    num_decoder_layers=hyperparameters['num_decoder_layers'],
    emb_dropout_prob=hyperparameters['emb_dropout_prob'],
    attn_dropout_prob=hyperparameters['attn_dropout_prob'],
    ff_dropout_prob=hyperparameters['ff_dropout_prob'],
    attn_use_bias=hyperparameters['attn_use_bias'],
    ff_use_bias=hyperparameters['ff_use_bias'],
    output_features_bias=hyperparameters['output_features_bias'],
)

### Number of parameters

In [ ]:
def print_model_parameters(model):
    print(f'{sum(p.numel() for p in model.parameters()):,}')

In [ ]:
print('Number of parameters:')
print_model_parameters(model)

In [ ]:
# "1" is the batch, single sample to speed it up
dummy_data = torch.randn(1, hyperparameters['window_size'], hyperparameters['input_features_size'])
dummy_date = torch.randn(1, hyperparameters['window_size'], hyperparameters['date_input_features_size'])

o = model(dummy_data, dummy_date)
o.shape

# FFT as Function

In [ ]:
def calculate_frequency_components(data, sampling_rate):
    fft_result = np.fft.fft(data)
    fft_freqs = np.fft.fftfreq(len(data), 1/sampling_rate)
    return fft_freqs, fft_result

In [ ]:
def inverse_transform(fft_result):
    # IFFT is complex!
    return np.fft.ifft(fft_results).real

# BandPass Filter

A bandpass filter allows signals within a specific frequency range to pass through while cutting off the frequencies outside that range
* Frequency Range: $[\text{low\_cutoff}, \text{high\_cutoff}]$

In [ ]:
def bandpass_filter(fft_result, low_cutoff=10.0, high_cutoff=1000.0):
    filtered_fft_results = np.zeros_like(fft_result)
    
    # pass only frequencies between low and high cutoff frequencies
    for i in range(len(fft_result)):
        if low_cutoff <= np.abs(fft_result[i]) <= high_cutoff:
            filtered_fft_results[i] = fft_result[i]

    return filtered_fft_results

# Frequency Features
### Calculate sampling rate from index

In [ ]:
avg_delta_time = df_full.index.diff(periods=1).dropna().total_seconds().to_numpy().mean()
SAMPLING_RATE = 1 / avg_delta_time

# NOTE: since dataset is hourly sampled, sampling rate is really low
# most of the time, this is not the case!
print(f'Sampling Rate: {SAMPLING_RATE} Hz')

In [ ]:
def frequency_featues(data_column, sampling_rate):
    fft_freqs, fft_results = calculate_frequency_components(data_column, sampling_rate)
    filtered_fft_results = bandpass_filter(fft_results)
    magnitude = np.abs(filtered_fft_results)
    return magnitude

# helper function for setting "sampling rate"
# ignore second of input tuple (values, datetime)
add_frequency_features = lambda data_column: frequency_featues(data_column, SAMPLING_RATE)

# Dataset

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, window_size, features_with_fft):
        self.df = df
        self.window_size = window_size
        self.features_with_fft = features_with_fft
    
    def __len__(self):
        number_of_samples = self.df.shape[0]
        # make sure that last window fits
        return number_of_samples - self.window_size

    def __getitem__(self, start_idx):

        # get a NumPy array of size: (hyperparameters['window_size'], NUM_FEATURES)
        df_window_original = self.df.iloc[start_idx:start_idx+self.window_size]

        df_window = df_window_original.copy()
        
        # FFT features
        for _feature_name in self.features_with_fft:
            fft_frequencies = add_frequency_features(df_window[f'{_feature_name}'].values)
            df_window.loc[:, f'{_feature_name}_fft'] = fft_frequencies
        
        sample_window = df_window.values
        # input (lag) timestamps
        sample_timestamp_lags = df_window[:-1].index
            
        # divide window into lags and forecast (shifted by 1)
        # first window_size-1 steps
        lags = sample_window[:-1, :]
        # last window_size-1 steps
        forecast = sample_window[1:, :]

        # convert to tensor
        lags = torch.tensor(lags, dtype=torch.float32)
        forecast = torch.tensor(forecast, dtype=torch.float32)
        
        # (lags, date_input_features_size)
        date = torch.tensor([sample_timestamp_lags.month, sample_timestamp_lags.day, sample_timestamp_lags.hour], dtype=torch.float32).permute(1, 0)
        
        return lags, forecast, date

In [ ]:
dataset_full = TimeSeriesDataset(
    df_full,
    hyperparameters['window_size'],
    features_with_fft=hyperparameters['features_with_fft'] # TODO: add features here that require FFT
)

In [ ]:
_lags, _forecast, _date = dataset_full[0]
_lags.shape, _forecast.shape, _date.shape

### Train/Test Split

In [ ]:
train_size = int(hyperparameters['split_ratio'] * len(dataset_full))
test_size = len(dataset_full) - train_size

train_size, test_size

In [ ]:
df_train = df_full.iloc[:train_size, :]
df_test = df_full.iloc[-test_size:, :]

# Airline Passgeners
#plt.plot(df_train.index, df_train['Passengers'], c='blue', label='training')
#plt.plot(df_test.index, df_test['Passengers'], c='red', label='test')

# Panama Electricity Load Forecasting
plt.plot(df_train.index, df_train['nat_demand'], c='blue', label='training')
plt.plot(df_test.index, df_test['nat_demand'], c='red', label='test')

plt.legend()

In [ ]:
dataset_train = TimeSeriesDataset(
    df_train,
    hyperparameters['window_size'],
    features_with_fft=hyperparameters['features_with_fft'] # TODO: add features here that require FFT
)

dataset_test = TimeSeriesDataset(
    df_test,
    hyperparameters['window_size'],
    features_with_fft=hyperparameters['features_with_fft'] # TODO: add features here that require FFT
)

# Dataloader

In [ ]:
dataloader_full = DataLoader(
    dataset_full,
    batch_size=hyperparameters['batch_size'],
    shuffle=False,
)

dataloader_train = DataLoader(
    dataset_train,
    batch_size=hyperparameters['batch_size'],
    shuffle=False,
)

dataloader_test = DataLoader(
    dataset_test,
    batch_size=hyperparameters['batch_size'],
    shuffle=False,
)

print(f'Number of batches (total): {len(dataloader_full)}')
print(f'Number of batches (train): {len(dataloader_train)}')
print(f'Number of batches (test): {len(dataloader_test)}')

In [ ]:
_lags_batch, _forecast_batch, _date_batch = next(iter(dataloader_full))
# (hyperparameters['batch_size'], hyperparameters['lags'], NUM_FEATURES), # (hyperparameters['batch_size'], hyperparameters['forecast'], NUM_FEATURES)
_lags_batch.shape, _forecast_batch.shape, _date_batch.shape

In [ ]:
len(dataloader_full)

# Training

### Training Functions
* with AMP support

In [ ]:
def train_iter(model, dataloader, optimizer, criterion, scaler, use_amp, device):
    model.train()

    avg_loss = []
    
    for (lags, forecast, date) in dataloader:

        lags = lags.to(device)
        forecast = forecast.to(device)
        date = date.to(device)
        
        with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
            # AMP forward pass
            forecast_pred = model(lags, date)
            loss = criterion(forecast_pred, forecast)
        
        scaler.scale(loss).backward()  #loss.backward()
        scaler.step(optimizer)         #optimizer.step()
        scaler.update()
        optimizer.zero_grad()

        avg_loss.append(loss.item())

    return sum(avg_loss) / len(avg_loss)


@torch.no_grad()
def eval_iter(model, dataloader, criterion, use_amp, device):
    model.eval()

    avg_loss = []
    predictions = []
    
    for (lags, forecast, date) in dataloader:
        
        lags = lags.to(device)
        forecast = forecast.to(device)
        date = date.to(device)
                
        with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
            # AMP forward pass
            forecast_pred = model(lags, date)
            loss = criterion(forecast_pred, forecast)
        
        avg_loss.append(loss.item())
        predictions.append(forecast_pred)

    return sum(avg_loss) / len(avg_loss), predictions

### Start Training

In [ ]:
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=hyperparameters['learning_rate']
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min', 
    factor=0.1, 
    patience=3,
    min_lr=1e-6,
)

scaler = torch.cuda.amp.GradScaler(enabled=hyperparameters['use_amp'])

mse_loss = nn.MSELoss()

In [ ]:
model.to(device)

for epoch in range(1, hyperparameters['num_epochs']+1):
        
    avg_train_loss = train_iter(
        model=model, 
        dataloader=dataloader_train, 
        optimizer=optimizer, 
        criterion=mse_loss, 
        scaler=scaler, 
        use_amp=hyperparameters['use_amp'], 
        device=device
    )
    
    avg_test_loss, _ = eval_iter(
        model=model, 
        dataloader=dataloader_test, 
        criterion=mse_loss, 
        use_amp=hyperparameters['use_amp'],
        device=device
    )

    # auto decrease LR when not improving
    scheduler.step(avg_test_loss)
    
    """
    # MANUAL LR SCHEDULING
    if epoch == 30:
        for pg in optimizer.param_groups:
            pg['lr'] *= 0.1

    if epoch == 60:
        for pg in optimizer.param_groups:
            pg['lr'] *= 0.1
    """
    
    print(f'Epoch: {epoch:<3}| Training loss: {avg_train_loss:.4f}, Testing Loss: {avg_test_loss:.4f}, LR: {scheduler.get_last_lr()}')

# Testing
* NOTE: we prefer to use single windows step during prediction

In [ ]:
@torch.no_grad()
def eval_iter_single_step(model, dataloader, criterion, use_amp, device):
    model.eval()

    avg_loss = []
    predictions = []
    
    # single step
    for start_idx in tqdm(range(0, len(df_test)-hyperparameters['window_size'])):

        # single step window sliding
        df_window_original = df_test.iloc[start_idx:start_idx+hyperparameters['window_size'], :]

        # add FFT features
        df_window = df_window_original.copy()
        
        for _feature_name in hyperparameters['features_with_fft']:
            fft_frequencies = add_frequency_features(df_window[f'{_feature_name}'].values)
            df_window.loc[:, f'{_feature_name}_fft'] = fft_frequencies
        
        sample_window = df_window.values
        # input (lag) timestamps
        sample_timestamp_lags = df_window[:-1].index

        # covnert to tensor
        lags = torch.tensor(sample_window[:-1], dtype=torch.float32, device=device)
        forecast = torch.tensor(sample_window[1:], dtype=torch.float32, device=device)
        # (lags, date_input_features_size)
        date = torch.tensor([sample_timestamp_lags.month, sample_timestamp_lags.day, sample_timestamp_lags.hour], dtype=torch.float32, device=device).permute(1, 0)
        
        # artificially add batch dimension
        # (we are not using the dataloader here!)
        lags = lags.unsqueeze(0)
        forecast = forecast.unsqueeze(0)
        date = date.unsqueeze(0)

        with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
            forecast_pred = model(lags, date)
            loss = criterion(forecast_pred, forecast)
            
        avg_loss.append(loss.item())
        # (batch, forecast, output_features_size)-> (1, window_size-1, output_features_size)
        # TAKE THE LAST PREDICTION STEP AS FORECAST!
        predictions.append(forecast_pred[0][-1].cpu().numpy())

    return sum(avg_loss) / len(avg_loss), predictions

In [ ]:
_, pred_sliding = eval_iter_single_step(
    model=model, 
    dataloader=dataloader_test, 
    criterion=mse_loss, 
    use_amp=hyperparameters['use_amp'],
    device=device
)

len(pred_sliding)

In [ ]:
df_full.columns

In [ ]:
sliding_results_dict = {}

pred_sliding_array = np.array(pred_sliding)

for feature_id, feature_key in enumerate(df_full.columns):
    sliding_results_dict[feature_key] = pred_sliding_array[:, feature_id]
        
df_sliding = pd.DataFrame(data=sliding_results_dict, index=df_test.index[:-hyperparameters['window_size']])

df_sliding.head(10)

In [ ]:
selected_feature = 'nat_demand'

df_train_plot = df_train[:-hyperparameters['window_size']].copy()
df_test_plot = df_test[:-hyperparameters['window_size']].copy()

plt.plot(df_train_plot.index, df_train_plot[selected_feature], c='blue', label='training')
plt.plot(df_test_plot.index, df_test_plot[selected_feature], c='red', label='test')
plt.plot(df_sliding.index, df_sliding[selected_feature] , c='green', label='prediction')
plt.legend()

In [ ]:
plt.figure(figsize=(10, 6))
df_test.loc['2019-01-03 05:00:00':'2019-01-12 05:00:00', 'nat_demand'].plot(label='actual')
df_sliding.loc['2019-01-03 05:00:00':'2019-01-12 05:00:00', 'nat_demand'].plot(label='prediction')
plt.legend()

# Save Model

In [ ]:
os.makedirs('./saved_models', exist_ok=True)

checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'hyperparameters': hyperparameters,
}

if hyperparameters['use_amp']:
    checkpoint['scaler_state_dict'] = scaler.state_dict()

torch.save(
    checkpoint,
    './saved_models/GPTTimeSeries_Autoregressive_FFT.pt'
)

# Generative Forecast

In [ ]:
@torch.no_grad()
def generative_forecast(model, data, timestamps, num_steps, lag_window_size, use_amp, device):
    model.eval()
    
    predictions = []
    time_indexes = []
    
    # covnert to tensor
    # data.shape: (lags, features)
    lags = torch.tensor(data[-lag_window_size:, :], dtype=torch.float32, device=device)
    
    # artificially add batch dimension
    # (we are not using the dataloader here!)
    # data.shape: (1, lags, features)
    lags = lags.unsqueeze(0)

    # Datetime indexes 
    #timestamps = df_full.index 
    # Delta time: calculate the time difference between two samples 
    delta_time = timestamps[1] - timestamps[0]
    # Get last timestamp
    current_timestamp = timestamps[-1]

    def generate_date_tensor(_timestamp, _lags, _device):
        _timestamp = _timestamp[-lag_window_size:]
        return torch.tensor([_timestamp.month, _timestamp.day, _timestamp.hour], dtype=torch.float32, device=_device).permute(1, 0)
    
    # single step
    for idx in tqdm(range(num_steps)):

        # get the last lag steps
        lags = lags[:, -lag_window_size:, :]
        #print(lags)

        # date
        date = generate_date_tensor(timestamps, lag_window_size, device).unsqueeze(0)

        with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
            forecast_pred = model(lags, date)
        
        # (batch, forecast, output_features_size)-> (1, window_size-1, output_features_size)
        # TAKE THE LAST PREDICTION STEP AS FORECAST!
        predictions.append(forecast_pred[0][-1].cpu().numpy())

        # update current timestamp
        current_timestamp = current_timestamp + delta_time
        time_indexes.append(current_timestamp)
        
        # append last forecast to the end
        # TAKE THE LAST PREDICTION STEP AS FORECAST!
        lags = torch.cat((lags, forecast_pred[:, -1:, :].detach()), dim=1)
        
        # next timestamp
        timestamps = timestamps + delta_time

    return predictions, time_indexes

In [ ]:
# should be min of original model
REQUEST_WINDOW_SIZE = 200 * 2 # * 2 is added for convenience
# temp dataframe for generative prediction input
df_temp_original = df_full[-REQUEST_WINDOW_SIZE:]

# Add FFT features
df_temp = df_temp_original.copy()

for _feature_name in hyperparameters['features_with_fft']:
    fft_frequencies = add_frequency_features(df_temp[f'{_feature_name}'].values)
    df_temp.loc[:, f'{_feature_name}_fft'] = fft_frequencies
        

pred_generative, time_indexes_generative = generative_forecast(
    model=model, 
    data=df_temp.values,
    timestamps=df_temp.index,
    num_steps=300, 
    lag_window_size=hyperparameters['window_size'], 
    use_amp=hyperparameters['use_amp'], 
    device=device
)

pred_generative_array = np.array(pred_generative)

generative_results_dict = {}

# loop ove features
for feature_id, feature_key in enumerate(df_temp.columns):
    generative_results_dict[feature_key] = pred_generative_array[:, feature_id]
        
df_generative = pd.DataFrame(data=generative_results_dict, index=time_indexes_generative)

# REVERSE THE PREPROCESSING FOR ORIGINAL RANGE
columns_without_fft = [c for c in df_generative.columns if not 'fft' in c]
df_generative[columns_without_fft] = feature_scaler.inverse_transform(df_generative[columns_without_fft])    

In [ ]:
df_generative[selected_feature].plot()